- Ollama -> See : https://ollama.com/download
  - For linux users :
    
    For this tutorial, run these instructions
    - Install Ollama : `curl -fsSL https://ollama.com/install.sh | sh`
    - Pull required models : 
        - `ollama pull mayflowergmbh/occiglot-7b-fr-en-instruct` #french llm model
        - `ollama pull sammcj/sfr-embedding-mistral:Q4_K_M` # decent embedding for this use case
    
    -------------------------------------------------------------------------------
  
    Additional informations about Ollama
    - To remove Ollama : https://github.com/ollama/ollama/blob/main/docs/linux.md
    - To stop ollama server : `systemctl stop ollama`
    - To restart server : `systemctl start ollama`
    
    -------------------------------------------------------------------------------
    Due to issues from Ollama latest versions in weigths update, we might want to install older versions of Ollama.
    To install **v.0.1.31** on Linux:
      - `curl -fsSL https://ollama.com/install.sh | sed 's#https://ollama.com/download/ollama-linux-${ARCH}${VER_PARAM}#https://github.com/ollama/ollama/releases/download/v0.1.31/ollama-linux-amd64#' | sh`
            
- Next steps
    - Add source for response provided by the chatbot (e.g. source : from 'Le Challenger' via 'Malijet'. To know more, here are some useful links: links...)
    - Improve model response (accuracy and precision)


In [59]:
from pathlib import Path
import os
import uuid
import pandas as pd
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores.chroma import Chroma
# from langchain_community.vectorstores import DocArrayInMemorySearch
# from langchain_openai.embeddings import OpenAIEmbeddings

In [41]:
%env OPENAI_API_KEY=sk-xxx

env: OPENAI_API_KEY=sk-xxx


In [2]:
ARTICLE_SOURCE_FILE_PATH = Path().resolve().parent /"data" / "malijet" / "source.csv"
CHROMA_DB_PERSIST_PATH = Path().resolve().parent / "data" / "chroma_db"
MODEL_NAME = "mayflowergmbh/occiglot-7b-fr-en-instruct"

In [3]:
# get nb cpu
os.cpu_count()

16

In [4]:
system_role = "Tu es un expert sur les actualités du Mali et tu parles uniquement français (spécialisé en langue française)."
llm = Ollama(model=MODEL_NAME, system=system_role, num_thread=os.cpu_count()-6)
llm

Ollama(model='mayflowergmbh/occiglot-7b-fr-en-instruct', num_thread=10, system='Tu es un expert sur les actualités du Mali et tu parles uniquement français (spécialisé en langue française).')

## Testing Simple LLM discussion

In [270]:
llm.invoke("Cite moi les noms des présidents du Mali.")

"Les présidents du Mali sont :\n- Modibo Keïta (1960-1968)\n- Moussa Traoré (1968-1991)\n- Amadou Toumani Touré (2002-2012)\n- Dioncounda Traoré (intérim, 2012-2013)\n- Ibrahim Boubacar Keïta (2013-2020)\n- Bah N'Daw (intérim, 2020)\n- Assimi Goïta (depuis 2020)."

In [271]:
llm.invoke("Quelle est la plus grande crise que la Mali a connue ?")

"La plus grande crise économique que le Mali a connue est la crise financière de 2012-2013, qui a été causée par une combinaison de facteurs tels que les chocs exogènes, la mauvaise gouvernance et la corruption. Cette crise a entraîné une forte dépréciation du franc CFA malien, une baisse des investissements étrangers et une augmentation de l'inflation. Le gouvernement malien a mis en place plusieurs mesures pour remédier à cette situation, notamment la réduction des dépenses publiques, la privatisation d'entreprises publiques et la mise en place de politiques économiques plus libérales. Cependant, le pays est encore confronté à de nombreux défis économiques, tels que la faible croissance économique, la pauvreté et l'instabilité politique."

In [276]:
llm.invoke("Who is the most popular scientist in the world?")

"Il n'y a pas de réponse définitive à cette question car la popularité est subjective et peut varier selon les individus et les régions. Cependant, certains scientifiques célèbres qui ont contribué de manière significative à leur domaine sont Albert Einstein, Stephen Hawking, Marie Curie, Isaac Newton et Charles Darwin."

## 2. Build RAG with CSV file

In [5]:
loader = CSVLoader(
    file_path=ARTICLE_SOURCE_FILE_PATH,
    csv_args={
        "delimiter": "\t"
    }
)

# load documents
data = loader.load()
data[:3] # three first documents

[Document(page_content='title: Fin de la transition militaire: des propositions farfelues du Panel des démocrates\nsource_paper: Info Matin\ndate: 2024-05-16\nlink: https://malijet.com/a_la_une_du_mali/290855-fin-de-la-transition-militaire-des-propositions-farfelues-du-pan.html\ncontent: Des supposés ‘’démocrates’’ maliens qui n’ont pas osé s’afficher dans leur communiqué de presse d’appel à la mobilisation pour la mise en place d’une transition civile auront-ils le courage de fouler le territoire national pour la concrétisation de leur projet ? La déclaration de ce ‘’Panel des démocrates’’ nous rappelle l’affaire de l’opposant malien, Ainea Ibrahim CAMARA, qui s’était proclamé président du Mali depuis la Côte d’Ivoire. Dans son document, le fameux ‘’Panel des démocrates’’ expose des propositions farfelues en demandant que le pays soit dirigé par un chef d’État honorifique désigné chaque année par exercice tournant parmi les sénateurs...Autre hic, alors que certains

In [6]:
# 53 articles extracted
len(data), pd.read_csv(ARTICLE_SOURCE_FILE_PATH, sep="\t").shape[0]

(53, 53)

## Let's see if the splitter is necessary

### Without splitter

In [7]:
def get_length_info(list_of_documents, splitters=None):
    
    if splitters is None:
        splitters = [' ', '.']
    
    ## search the content length statistics (nb characters)
    print('-'*10, "For character length", '-'*10)
    display(pd.Series([len(document.page_content) for document in list_of_documents]).describe())
    
    ## search the words length statistics (nb words)
    print('-'*10, "Length of words (nb characters) in the corpus", '-'*10)
    _res = list()
    for document in list_of_documents:
        _res += pd.Series(document.page_content.split(splitters[0])).apply(len).tolist()
    display(pd.Series(_res).describe())
    
    ## search the sentence length statistics (nb words)
    print('-'*10, "How many words in each doc", '-'*10)
    display(pd.Series([len(doc.page_content.split(splitters[0])) for doc in list_of_documents]).describe())
    
    ## search the sentence length statistics (nb words)
    print('-'*10, "For how many sentences in in each doc", '-'*10)
    display(pd.Series([len(doc.page_content.split(splitters[1])) for doc in list_of_documents]).describe())
    
    ## search the sentence length statistics (nb words)
    print('-'*10, "For sentences length in each doc", '-'*10)
    display(pd.Series([len(sentence) for document in list_of_documents for sentence in document.page_content.split('.')]).describe())
    
    return _res

In [8]:
res = get_length_info(data)

---------- For character length ----------


count       53.000000
mean      5078.716981
std       5284.925774
min        373.000000
25%       2038.000000
50%       3816.000000
75%       6236.000000
max      30532.000000
dtype: float64

---------- Length of words (nb characters) in the corpus ----------


count    40780.000000
mean         5.601888
std          5.527599
min          0.000000
25%          2.000000
50%          4.000000
75%          8.000000
max        123.000000
dtype: float64

---------- How many words in each doc ----------


count      53.000000
mean      769.433962
std       826.540415
min        30.000000
25%       304.000000
50%       571.000000
75%       931.000000
max      4616.000000
dtype: float64

---------- For how many sentences in in each doc ----------


count     53.000000
mean      31.660377
std       40.456027
min        4.000000
25%       14.000000
50%       21.000000
75%       35.000000
max      287.000000
dtype: float64

---------- For sentences length in each doc ----------


count    1678.000000
mean      159.443981
std       150.289294
min         0.000000
25%        76.000000
50%       132.000000
75%       207.000000
max      2386.000000
dtype: float64

## With splitter

In [9]:
pd.Series(res).max(), pd.Series(res).quantile(.99)

(123, 16.0)

In [10]:
quantile = int(pd.Series([len(sentence) for document in data for sentence in document.page_content.split('.')]).quantile(.95))
maxi_character_per_chunk = pd.Series([len(sentence) for document in data for sentence in document.page_content.split('.')]).max().astype(int)
maxi_character_per_words = 20 # over 16 to make sure to get all content overlap
quantile, maxi_character_per_chunk, maxi_character_per_words

(387, 2386, 20)

How do I determine how many characters form a meaningful chunk (i.e. understandable and self-sufficient information) ?
We will assume that each sentence could be a meaningful chunk. The idea is to have a count distribution of sentences length in our courpus to make a quick decision.
- We might choose 400 (similar to the quantile q95) as chunk size to be more flexible.
- Also, in this case, we'll set 20 as the maximum length of character possible in a sentence

In [11]:
## test the document splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=quantile, # selected quantile
    chunk_overlap=maxi_character_per_words,
    separators=["\n\n", "\n", ". ", " ", ""], # specify that sentence split (by dot) is more important than space & other
    keep_separator=False # drop the separators from the document after split
)
documents = text_splitter.split_documents(documents=data)
len(documents)

962

In [12]:
documents[8:20]

[Document(page_content='Sans ambages, ce panel des démocrates appelle à un soulèvement contre la transition en cours en vue de rétablir « la souveraineté populaire et l’ordre constitutionnel, à soutenir l’investiture du gouvernement de transition en exil et à participer massivement à la manifestation populaire des démocrates maliens le 28 juin 2024 à Bamako, dans les autres localités du Mali et à', metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 0}),
 Document(page_content='du Mali et à l’étranger..', metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 0}),
 Document(page_content='». Dernière ce projet machiavélique qui n’est pas visiblement soutenu par certains hommes politiques joints par nos soins, malgré leur divergence avec les autorités de la transition, se cache le désir de ces ‘’démocrates’’ de vouloir mettre en place un gouvernement en exil pour agir au nom et au compte d’un pay

In [13]:
res = get_length_info(documents)

---------- For character length ----------


count    962.000000
mean     279.697505
std       93.495508
min       14.000000
25%      229.000000
50%      300.000000
75%      356.000000
max      387.000000
dtype: float64

---------- Length of words (nb characters) in the corpus ----------


count    40826.000000
mean         5.614192
std          5.277430
min          0.000000
25%          2.000000
50%          4.000000
75%          8.000000
max        118.000000
dtype: float64

---------- How many words in each doc ----------


count    962.000000
mean      42.438669
std       15.873750
min        1.000000
25%       32.000000
50%       46.000000
75%       55.000000
max       79.000000
dtype: float64

---------- For how many sentences in in each doc ----------


count    962.000000
mean       1.918919
std        1.287770
min        1.000000
25%        1.000000
50%        2.000000
75%        2.000000
max       12.000000
dtype: float64

---------- For sentences length in each doc ----------


count    1846.000000
mean      145.278982
std       105.502656
min         0.000000
25%        69.000000
50%       127.500000
75%       205.000000
max       387.000000
dtype: float64

 We move from 4616 words maxi per doc to 79 words per doc. That might be relevant for the final framework of RAG.

In [14]:
[doc.page_content for doc in documents[:5]]

['title: Fin de la transition militaire: des propositions farfelues du Panel des démocrates\nsource_paper: Info Matin\ndate: 2024-05-16\nlink: https://malijet.com/a_la_une_du_mali/290855-fin-de-la-transition-militaire-des-propositions-farfelues-du-pan.html',
 'content: Des supposés ‘’démocrates’’ maliens qui n’ont pas osé s’afficher dans leur communiqué de presse d’appel à la mobilisation pour la mise en place d’une transition civile auront-ils le courage de fouler le territoire national pour la concrétisation de leur projet ? La déclaration de ce ‘’Panel des démocrates’’ nous rappelle l’affaire de l’opposant malien, Ainea Ibrahim',
 'Ainea Ibrahim CAMARA, qui s’était proclamé président du Mali depuis la Côte d’Ivoire',
 'Dans son document, le fameux ‘’Panel des démocrates’’ expose des propositions farfelues en demandant que le pays soit dirigé par un chef d’État honorifique désigné chaque année par exercice tournant parmi les sénateurs...Autre hic, alors que certai

In [15]:
[doc.page_content for doc in documents[37:40]]

['peuvent être mises à profit pour l’émergence d’une expertise endogène ; la promotion de l’amour de la patrie et l’esprit du don de soi ; la recherche des mécanismes innovants pour soutenir l’effort de guerre ; l’anticipation et l’accompagnement pour le retour des réfugiés et des déplacés dans leurs localités respectives ; la prise des dispositions nécessaires pour la',
 'nécessaires pour la réinsertion socio-économique des déplacés et des réfugiés',
 'Doter le pays d’un satellite, sanctionner les fausses informations, créer des comités de veille citoyenne Parmi lesdites recommandations figurent la prise en compte de nos valeurs sociétales dans la gestion des crises ; la dotation du Mali d’un satellite de communication et de surveillance pouvant couvrir tout le territoire ; rendre la justice, promouvoir le pardon et le consensus']

## Embeddings and Vector store

In [16]:
# embeddings_llm = OllamaEmbeddings(model=MODEL_NAME) #mistral or Occiglot
# embeddings_llm = OllamaEmbeddings(model="mayflowergmbh/occiglot-7b-fr-en-instruct")
# embeddings_llm = OllamaEmbeddings(model="snowflake-arctic-embed")
# embeddings_llm = OpenAIEmbeddings()

embeddings_llm = OllamaEmbeddings(model="sammcj/sfr-embedding-mistral:Q4_K_M")

# Set few params if needed
embeddings_llm.show_progress = True
embeddings_llm.num_thread = os.cpu_count() - 8 # 10 in my case
# embeddings_llm.top_k = 10
# embeddings_llm.top_p = .5

embeddings_llm

OllamaEmbeddings(base_url='http://localhost:11434', model='sammcj/sfr-embedding-mistral:Q4_K_M', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=8, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=True, headers=None, model_kwargs=None)

In [17]:
# Exemple
documents[3], len(documents[3].page_content)

(Document(page_content='Dans son document, le fameux ‘’Panel des démocrates’’ expose des propositions farfelues en demandant que le pays soit dirigé par un chef d’État honorifique désigné chaque année par exercice tournant parmi les sénateurs...Autre hic, alors que certains démocrates exigent le retour immédiat à l’ordre constitutionnel, eux ils proposent encore une autre transition dite civile de', metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 0}),
 386)

In [25]:
embeddings_llm.embed_query(documents[3].page_content)[:7], len(embeddings_llm.embed_query(documents[3].page_content))

OllamaEmbeddings: 100%|██████████| 1/1 [00:14<00:00, 14.85s/it]


([2.0487639904022217,
  7.367893695831299,
  2.412371873855591,
  2.96880841255188,
  5.586341857910156,
  -13.39712905883789,
  -2.1971967220306396],
 4096)

## Selection of Vector Store

Thanks to this brand-new article from Google, we can safely choose any open source Vector Store, make it available to Google NFS Filestore and access it easily through mounting filestore in Cloud Run (see section 3): https://cloud.google.com/blog/products/serverless/introducing-cloud-run-volume-mounts?hl=en

In [36]:
# The embedding is very large, dimension of 4096, so this will take a real long time
# Database creation
db = Chroma.from_documents(
    documents=documents[:2],
    embedding=embeddings_llm,
    persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(),
)
db

OllamaEmbeddings: 100%|██████████| 2/2 [00:32<00:00, 16.49s/it]


In [122]:
db = Chroma(
    persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(),
    embedding_function=embeddings_llm,
)

persisted_ids = db.get()["ids"]

new_documents_to_embed_df = pd.DataFrame({
    "single_id": [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in documents],
    "document": documents
})

# to keep only different documents (i.e. chunks)
new_documents_to_embed_df.drop_duplicates(subset="single_id", inplace=True)

# Keep only documents not already embedded
new_documents_to_embed_df = new_documents_to_embed_df.query(f"single_id not in {persisted_ids}")

if new_documents_to_embed_df.empty:
    print("No documents to embed")
else:
    print("Embedding documents...")
    display(new_documents_to_embed_df.head())
    db.add_documents(
        documents=new_documents_to_embed_df.document.tolist(), 
        embedding=embeddings_llm, 
        ids=new_documents_to_embed_df.single_id.tolist(), 
        persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(),
    )


No documents to embed


In [121]:
db_check = Chroma(
    persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(),
    embedding_function=embeddings_llm,
)
# db_check.get()

In [42]:
# db_check.get(include=['embeddings'])

In [62]:
# db_check.delete_collection()

In [23]:
# In memory vector store test !

# Took more than 36min for 46 documents only; wow, I interrupted with the keyboard!
# For model name as occiglot or mistral or sfr-embedding first on retrieval
# db = DocArrayInMemorySearch.from_documents(documents, embedding=embeddings_llm)


OllamaEmbeddings: 100%|██████████| 962/962 [2:28:09<00:00,  9.24s/it]


In [171]:
# for snowflake
# db2 = DocArrayInMemorySearch.from_documents(documents, embedding=embeddings_llm) 

OllamaEmbeddings: 100%|██████████| 219/219 [03:31<00:00,  1.04it/s]


In [88]:
embeddings_llm

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7f8b52977700>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7f8b511c1360>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [36]:
# for openai
# db4 = DocArrayInMemorySearch.from_documents(documents, embedding=embeddings_llm)

In [11]:
# load db from disk
# db3 = Chroma(persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(), embedding_function=embeddings_llm)
# db3

In [19]:
llm.invoke("Tu connais le Mali ?")

"\nCet article sera écrit en anglais. Si vous préférez lire en français, voir notre autre article sur le Mali : <https://www.toutlemondeenligne.com/fr/mali>.\n\nThe Mali is a landlocked country located in West Africa. It has a population of approximately 18 million people and its official language is French, although Bambara, the most widely spoken local language, is also used in everyday life. The capital city of Mali is Bamako, which is located in the central part of the country.\n\nMali is known for its rich history and culture, as well as its natural beauty. Some popular attractions include the Dogon region, which is home to ancient rock formations and a unique way of life, and Timbuktu, an historic city that was once a major center of trade and learning in Africa.\n\nMali is also known for its beautiful landscapes, including vast savannas, rolling hills, and lush forests. Wildlife enthusiasts can enjoy activities such as bird watching, hiking, and camping in the country's national

## Retriever for RAG

In [37]:
retriever = db.as_retriever(search_kwargs={"k": 10}) # Occiglot or sfr-embed
# retriever = db2.as_retriever(search_kwargs={"k": 10}) # snowflake
# retriever = db4.as_retriever(search_kwargs={"k": 10}) #openAI
# retriever = db3.as_retriever(search_kwargs={"k": 5})
retriever

VectorStoreRetriever(tags=['DocArrayInMemorySearch'], vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x74d9a69c96f0>, search_kwargs={'k': 10})

In [173]:
query = "AES" # snowflake older version
retriever.invoke(query), len(retriever.invoke(query))

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  6.69it/s]


([Document(page_content='l’Alliance des États du Sahel (AES)', metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 2}),
  Document(page_content='mutuelle', metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 2}),
  Document(page_content='mai', metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 0}),
  Document(page_content='»', metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 0}),
  Document(page_content='des principes démocratiques, entre autres', metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 0}),
  Document(page_content='pour la paix et la réconciliation nationale', metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 5}),
  Document(page_content='S. Guindo/Malijet.com', metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.cs

In [38]:
# Prompt template
template = """
Réponds à la question uniquement grâce au contexte suivant et uniquement en langue française.
Si tu n'as pas de réponse explicite dans le contexte, réponds "Je n'ai pas assez d'informations pour répondre correctement".

Contexte : {context}

Question : {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [39]:
def format_docs(docs):
    print([d.page_content for d in docs])
    return "\n\n".join([d.page_content for d in docs])

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [30]:
questions = [
    # "Qui est Oumar Diarra ?",
    # "Quels sont les actions de l'EUTM ?",
    # "Cite moi les recommandations retenues lors du dialogue inter malien",
    # "Quand finit la mission de l'union européenne ?",
    # "Actualités sur la DIRPA",
    # "Parle moi de l'agriculture au Mali",
    # "Que font les FAMA actuellement ?"
    
    "Parle moi du nouveau vérificateur général",
    "Résume moi en quelques points les dernières actualités maintenant",
    "Où en est la relation Mali et Russie ?",
    "Qui est Bassaro Haïdara ?",
    "Qu'est ce que Assimi a fait récemment ?",
    "Le dialogie inter malien est il terminé ?",
    "Qui sont les membres de l'AES ?",
    "Comment a été la journée du 1er Mai au Mali ?",
    "Donne moi la date la plus récente des informations dont tu disposes",
    "Qu'en est il de la crise sécuritaire au Mali ?",
    "La Belgique a t elle récemment collaborée avec le Mali ?" # question bonus (must return I dont know)
]
len(questions)

11

In [31]:
# for open source models
for q in questions:
    print(q)
    print(chain.invoke(q))
    print('-'*50, "\n")

Parle moi du nouveau vérificateur général



OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.62s/it]


['L’audience était présidée par le vice-président de la Cour suprême, Moussa Diarra, qui avait à ses côtés  Hamadoun Cissé et Tiécoura Mallé. A l’ouverture de la séance, le greffier a lu la décision du Vérificateur général, Samba Alhamdou Baby, portant nomination du nouveau vérificateur', 'et de Jeunes et les autorités administratives indépendantes', 'Les conclusions qui fâchent sont relatives à la nouvelle prorogation de la transition, à la relecture de la Charte de la transition, à l’appel à la candidature de Assimi GOITA, à l’élévation des jeunes officiers au grade du général', 'la corruption ; l’affichage de tous les tarifs des actes et prestations dans ces domaines et la mise en place rapide de la législation sur la protection des lanceurs d’alerte', 'content: Il est 9 heures lorsque Mohamed Daou, Vérificateur spécialisé en marchés publics au Bureau du Vérificateur général est appelé à la barre où il doit prêter serment devant la Cour suprêm


OllamaEmbeddings: 100%|██████████| 1/1 [00:11<00:00, 11.47s/it]


['Ces braves hommes et femmes en tenue ont survécu, ont approché la mort, ont été attaqués et ont patienté de longues années avant que leur mission ne passe pour prioritaire aujourd’hui', 'ceux et celles qui n’ont pas pris part au Dialogue Directe inter-maliens de les rejoindre', 'dont nous vivons les premiers instants ici et ensemble', 'Ces pourparlers directs sans ingérence extérieure visent à permettre des solutions endogènes à la crise que traverse le pays depuis plus d’une décennie. Et depuis cette annonce, les choses sont allées vite et même très vite', 'Ces installations modernes sont conçues pour répondre aux standards internationaux et visent à renforcer l’autosuffisance du Mali dans ces secteurs clés. La mise en service de ce complexe industriel est imminente, et Seydou KEÏTA a assuré que les portes s’ouvriraient très prochainement', 'Jiddou dit Zeidane et Moustaph Koné respectivement directeur et animateur de la radio Coton d’Ansongo dont nous sommes encore 

...

['Tout a été dit, tout a été noté et consigné dans les rapports réceptionnés par une Commission de centralisation au sein du Comité de Pilotage et reversés aux commissions thématiques pour traitement et synthétisation, afin de mettre à la disposition des délégués à la Phase nationale des documents cohérents et consolidés', 'ceux et celles qui n’ont pas pris part au Dialogue Directe inter-maliens de les rejoindre', 'Cela nécessiterait aussi qu’il y ait une forte solidarité entre les hommes de média, notamment pour leur sécurité physique, morale et aussi économique. Le troisième niveau, c’est celui des citoyens. Ils ne doivent pas se contenter d’une information qui ne serait pas fiable', 'de haute intensité, c’est à vous, Chers Officiers militaires, de prendre ça en compte', 'Ces installations modernes sont conçues pour répondre aux standards internationaux et visent à renforcer l’autosuffisance du Mali dans ces secteurs clés. La mise en service de ce complex


OllamaEmbeddings: 100%|██████████| 1/1 [00:05<00:00,  5.70s/it]


['Elle vient s’ajouter à d’autres initiatives de lutte contre le terrorisme comme le bataillon autonome des forces spéciales et des Centres d’aguerrissement. La Fira n’a pas été créée pour des actions subversives. Il ne faut pas que les Maliens voient le mal partout”, nous a expliqué un haut cadre de l’armée.', 'Il leur reste de relever le défi de l’organisation des élections qui sont en instance, pour que le concept de Mali Kura démarre avec la réalité du pouvoir légitime et populaire transmis à un des futurs prochains nombreux candidats gagnant à l’élection présidentielle de fin 2024 ou début 2025. (C’est une prédiction, ce n’est pas officiel)', 'Heureusement qu’il y a eu plus de peur que de mal. Le directeur du CicB s’est personnellement investi à ce que tout soit bien rangé dans la salle. Après avoir regagné sa place au présidium, quelques minutes plus tard,  Pinochet a déclaré d’une voix grave : «Mesdames et Messieurs, bonjour. Nous arrivons au début de


OllamaEmbeddings: 100%|██████████| 1/1 [00:08<00:00,  8.13s/it]


['Elle vient s’ajouter à d’autres initiatives de lutte contre le terrorisme comme le bataillon autonome des forces spéciales et des Centres d’aguerrissement. La Fira n’a pas été créée pour des actions subversives. Il ne faut pas que les Maliens voient le mal partout”, nous a expliqué un haut cadre de l’armée.', 'content: Du 15 au 17 mai 2024, les travaux de la 1ère édition du colloque international sur le genre et la corruption se sont déroulés, à l’hôtel Radisson Collection (ex Sheraton) avec comme thème : « FEMME ET CORRUPTION : BRISER LE SILENCE ET AGIR ENSEMBLE ! »', 'Il leur reste de relever le défi de l’organisation des élections qui sont en instance, pour que le concept de Mali Kura démarre avec la réalité du pouvoir légitime et populaire transmis à un des futurs prochains nombreux candidats gagnant à l’élection présidentielle de fin 2024 ou début 2025. (C’est une prédiction, ce n’est pas officiel)', 'Bureau de l’Information et de la Presse/MAECI.', 'V

In [32]:
retriever.invoke("AES")


OllamaEmbeddings: 100%|██████████| 1/1 [00:05<00:00,  5.56s/it]


[Document(page_content='Renforcer l’indépendance et la transparence de l’institution judiciaire pour une meilleure distribution de la justice ; 23. Opérationnaliser la Cour des comptes avec la création de ses chambres régionales ; 24. Accélérer les procédures judiciaires ; 25. Elaborer et faire respecter les manuels de procédures dans les services de l’Etat ; 26', metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 14}),
 Document(page_content='Vulgariser et enseigner les chartes qui ont régi nos Royaumes et Empires ; 18. Développer, multiplier, diversifier, privilégier, pérenniser les accords de partenariats gagnant- gagnant ; 19. Établir une forte coopération entre les Forces de Défense de Sécurité (FDS) des pays de l’AES ; 20', metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 14}),
 Document(page_content='Instaurer un mécanisme de redevabilité et de transparence dans la gestion des ressources publiques ; 

## Récapitulatif

- 8 bons retrieval sur 11 questions [V, V, V, X, V, V, X, V, X, V, V] : V -> True response and X -> Wrong response
- 4 bonnes réponses sur 11 questions [X, V, V, X, V, X, X, X, X, X, V] :  V -> True response and X -> Wrong response
- Bonne réponse à la question Bonus !
- Questions potentielles à rajouter :
    - Quand est ce que Abdoulaye Diop rencontrera ses homologues ?
    -  Quoi de prévu le 28 Juin à Bamako ?
    - Quelle célébrité le président a rencontré ?
    - A quand la transition civile au Mali ?

# Some results from OpenAIEmbeddings,far better result

In [40]:
# Result from OpenAI Embeddings
for q in questions:
    print(q)
    print(chain.invoke(q))
    print('-'*50, "\n")

Parle moi du nouveau vérificateur général
['title: BVG : Le nouveau vérificateur Mohamed Daou prête serment\nsource_paper: Mali Tribune\ndate: 2024-05-02\nlink: https://malijet.com/a_la_une_du_mali/290367-bvg--le-nouveau-verificateur-mohamed-daou-prete-serment.html', 'content: Il est 9 heures lorsque Mohamed Daou, Vérificateur spécialisé en marchés publics au Bureau du Vérificateur général est appelé à la barre où il doit prêter serment devant la Cour suprême. C’était en présence de ses collègues, de ses proches et du directeur de la formation, de la documentation et du personnel, Drissa Mamadou Coulibaly', 'L’audience était présidée par le vice-président de la Cour suprême, Moussa Diarra, qui avait à ses côtés  Hamadoun Cissé et Tiécoura Mallé. A l’ouverture de la séance, le greffier a lu la décision du Vérificateur général, Samba Alhamdou Baby, portant nomination du nouveau vérificateur', 'Daou a juré devant la Cour, “de remplir ses fonctions avec h